The concept of our code (This is our first attempt at doing any moderate level coding on Python so please hold all laughing until the end of the document) is to demonstrate that the development of plays can alter the success or failure of defensive schemes. Specifically, we found that quarterbacks who can release the ball in under 3 seconds tend to complete 8 and 10 yard passes at a higher rate than QB's who lag in this category. Additionally, deep balls (Throws that exceed 15 yards) need at least 3 seconds to develop a route that puts the offensive at an advantage and defense at a disadvantage. Since the data was not available, we recommend that NFL Team Analytic Deparments utilize the Amazon Web Services (AWS) QB release data to optimize defensive schemes. Specifically, it seems that the Tampa 2 and Fairbanks-Bullough 3–4 defensive systems allow quick release QB's to have utility in completing a large amount of 8 to 10 yard passes. To assess release, we utilized the data to convert the play data into MP4 and manually gauged a wide variety of passes (All videos are not shown to shorten the notebook) regarding QB release (Which was relatively simple with 10 Frames Per Second Speed Setting). 

In [ ]:
import pandas as pd
import numpy as np
import os
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.patches as patches
pd.set_option('max_columns', 100)
from ipywidgets import interact, fixed
%matplotlib inline
from matplotlib import animation
from matplotlib.animation import FFMpegWriter
import dateutil
from math import radians
from IPython.display import Video
import warnings
warnings.filterwarnings('ignore')

Games = pd.read_csv('../input/nfl-big-data-bowl-2021/games.csv', low_memory=False)
Plays = pd.read_csv('../input/nfl-big-data-bowl-2021/plays.csv')
Players = pd.read_csv('../input/nfl-big-data-bowl-2021/players.csv')
Week1 = pd.read_csv('../input/nfl-big-data-bowl-2021/week1.csv')
Week2 = pd.read_csv('../input/nfl-big-data-bowl-2021/week2.csv')
Week3 = pd.read_csv('../input/nfl-big-data-bowl-2021/week3.csv')
Week4 = pd.read_csv('../input/nfl-big-data-bowl-2021/week4.csv')
Week5 = pd.read_csv('../input/nfl-big-data-bowl-2021/week5.csv')
Week6 = pd.read_csv('../input/nfl-big-data-bowl-2021/week6.csv')
Week7 = pd.read_csv('../input/nfl-big-data-bowl-2021/week7.csv')
Week8 = pd.read_csv('../input/nfl-big-data-bowl-2021/week8.csv')
Week9 = pd.read_csv('../input/nfl-big-data-bowl-2021/week9.csv')
Week10 = pd.read_csv('../input/nfl-big-data-bowl-2021/week10.csv')
Week11 = pd.read_csv('../input/nfl-big-data-bowl-2021/week11.csv')
Week12 = pd.read_csv('../input/nfl-big-data-bowl-2021/week12.csv')
Week13 = pd.read_csv('../input/nfl-big-data-bowl-2021/week13.csv')
Week14 = pd.read_csv('../input/nfl-big-data-bowl-2021/week14.csv')
Week15 = pd.read_csv('../input/nfl-big-data-bowl-2021/week15.csv')
Week16 = pd.read_csv('../input/nfl-big-data-bowl-2021/week16.csv')
Week17 = pd.read_csv('../input/nfl-big-data-bowl-2021/week17.csv')

In [ ]:
for col in Plays.columns: 
    print(col) 

In [ ]:
def create_football_field(linenumbers=True,
                          endzones=True,
                          highlight_line=False,
                          highlight_line_number=55,
                          highlight_first_down_line=False,
                          yards_to_go=10,
                          highlighted_name='Line of Scrimmage',
                          fifty_is_los=False,
                          figsize=(12, 6.33)):
    """
    Function that plots the football field for viewing plays.
    Allows for showing or hiding endzones.
    """
    rect = patches.Rectangle((0, 0), 120, 53.3, linewidth=0.1,
                             edgecolor='r', facecolor='darkgreen', zorder=0)

    fig, ax = plt.subplots(1, figsize=figsize)
    ax.add_patch(rect)

    plt.plot([10, 10, 10, 20, 20, 30, 30, 40, 40, 50, 50, 60, 60, 70, 70, 80,
              80, 90, 90, 100, 100, 110, 110, 120, 0, 0, 120, 120],
             [0, 0, 53.3, 53.3, 0, 0, 53.3, 53.3, 0, 0, 53.3, 53.3, 0, 0, 53.3,
              53.3, 0, 0, 53.3, 53.3, 0, 0, 53.3, 53.3, 53.3, 0, 0, 53.3],
             color='white')
    if fifty_is_los:
        plt.plot([60, 60], [0, 53.3], color='gold')
        plt.text(62, 50, '<- Player Yardline at Snap', color='gold')
    # Endzones
    if endzones:
        ez1 = patches.Rectangle((0, 0), 10, 53.3,
                                linewidth=0.1,
                                edgecolor='r',
                                facecolor='blue',
                                alpha=0.2,
                                zorder=0)
        ez2 = patches.Rectangle((110, 0), 120, 53.3,
                                linewidth=0.1,
                                edgecolor='r',
                                facecolor='blue',
                                alpha=0.2,
                                zorder=0)
        ax.add_patch(ez1)
        ax.add_patch(ez2)
    plt.xlim(0, 120)
    plt.ylim(-5, 58.3)
    plt.axis('off')
    if linenumbers:
        for x in range(20, 110, 10):
            numb = x
            if x > 50:
                numb = 120 - x
            plt.text(x, 5, str(numb - 10),
                     horizontalalignment='center',
                     fontsize=20,  # fontname='Arial',
                     color='white')
            plt.text(x - 0.95, 53.3 - 5, str(numb - 10),
                     horizontalalignment='center',
                     fontsize=20,  # fontname='Arial',
                     color='white', rotation=180)
    if endzones:
        hash_range = range(11, 110)
    else:
        hash_range = range(1, 120)

    for x in hash_range:
        ax.plot([x, x], [0.4, 0.7], color='white')
        ax.plot([x, x], [53.0, 52.5], color='white')
        ax.plot([x, x], [22.91, 23.57], color='white')
        ax.plot([x, x], [29.73, 30.39], color='white')

    if highlight_line:
        hl = highlight_line_number + 10
        plt.plot([hl, hl], [0, 53.3], color='yellow')
        #plt.text(hl + 2, 50, '<- {}'.format(highlighted_name),
        #         color='yellow')
        
    if highlight_first_down_line:
        fl = hl + yards_to_go
        plt.plot([fl, fl], [0, 53.3], color='yellow')
        #plt.text(fl + 2, 50, '<- {}'.format(highlighted_name),
        #         color='yellow')
    return fig, ax
create_football_field()
plt.show()

In [ ]:
Week1.head()

In [ ]:
fig, ax = create_football_field()
Week1.query("playId == 75 and team == 'away'") \
    .plot(x='x', y='y', kind='scatter', ax=ax, color='orange', s=30, legend='Away')
Week1.query("playId == 75 and team == 'home'") \
    .plot(x='x', y='y', kind='scatter', ax=ax, color='blue', s=30, legend='Home')
plt.title('Play # 75')
plt.legend()
plt.show()

In [ ]:
def calculate_dx_dy_arrow(x, y, angle, speed, multiplier):
    if angle <= 90:
        angle = angle
        dx = np.sin(radians(angle)) * multiplier * speed
        dy = np.cos(radians(angle)) * multiplier * speed
        return dx, dy
    if angle > 90 and angle <= 180:
        angle = angle - 90
        dx = np.sin(radians(angle)) * multiplier * speed
        dy = -np.cos(radians(angle)) * multiplier * speed
        return dx, dy
    if angle > 180 and angle <= 270:
        angle = angle - 180
        dx = -(np.sin(radians(angle)) * multiplier * speed)
        dy = -(np.cos(radians(angle)) * multiplier * speed)
        return dx, dy
    if angle > 270 and angle <= 360:
        angle = 360 - angle
        dx = -np.sin(radians(angle)) * multiplier * speed
        dy = np.cos(radians(angle)) * multiplier * speed
        return dx, dy
    
        
def animate_player_movement(weekNumber, playId, gameId):
    weekData = pd.read_csv('../input/nfl-big-data-bowl-2021/week' + str(weekNumber) + '.csv')
    playData = pd.read_csv('../input/nfl-big-data-bowl-2021/plays.csv')
    
    playHome = weekData.query('gameId==' + str(gameId) + ' and playId==' + str(playId) + ' and team == "home"')
    playAway = weekData.query('gameId==' + str(gameId) + ' and playId==' + str(playId) + ' and team == "away"')
    playFootball = weekData.query('gameId==' + str(gameId) + ' and playId==' + str(playId) + ' and team == "football"')
    
    playHome['time'] = playHome['time'].apply(lambda x: dateutil.parser.parse(x).timestamp()).rank(method='dense')
    playAway['time'] = playAway['time'].apply(lambda x: dateutil.parser.parse(x).timestamp()).rank(method='dense')
    playFootball['time'] = playFootball['time'].apply(lambda x: dateutil.parser.parse(x).timestamp()).rank(method='dense')
    
    maxTime = int(playAway['time'].unique().max())
    minTime = int(playAway['time'].unique().min())
    
    yardlineNumber = playData.query('gameId==' + str(gameId) + ' and playId==' + str(playId))['yardlineNumber'].item()
    yardsToGo = playData.query('gameId==' + str(gameId) + ' and playId==' + str(playId))['yardsToGo'].item()
    absoluteYardlineNumber = playData.query('gameId==' + str(gameId) + ' and playId==' + str(playId))['absoluteYardlineNumber'].item() - 10
    playDir = playHome.sample(1)['playDirection'].item()
    
    if (absoluteYardlineNumber > 50):
        yardlineNumber = 100 - yardlineNumber
    if (absoluteYardlineNumber <= 50):
        yardlineNumber = yardlineNumber
        
    if (playDir == 'left'):
        yardsToGo = -yardsToGo
    else:
        yardsToGo = yardsToGo
    
    fig, ax = create_football_field(highlight_line=True, highlight_line_number=yardlineNumber, highlight_first_down_line=True, yards_to_go=yardsToGo)
    playDesc = playData.query('gameId==' + str(gameId) + ' and playId==' + str(playId))['playDescription'].item()
    plt.title(f'Game # {gameId} Play # {playId} \n {playDesc}')
    
    def update_animation(time):
        patch = []
        
        homeX = playHome.query('time == ' + str(time))['x']
        homeY = playHome.query('time == ' + str(time))['y']
        homeNum = playHome.query('time == ' + str(time))['jerseyNumber']
        homeOrient = playHome.query('time == ' + str(time))['o']
        homeDir = playHome.query('time == ' + str(time))['dir']
        homeSpeed = playHome.query('time == ' + str(time))['s']
        patch.extend(plt.plot(homeX, homeY, 'o',c='gold', ms=20, mec='white'))
        
        # Home players' jersey number 
        for x, y, num in zip(homeX, homeY, homeNum):
            patch.append(plt.text(x, y, int(num), va='center', ha='center', color='black', size='medium'))
            
        # Home players' orientation
        for x, y, orient in zip(homeX, homeY, homeOrient):
            dx, dy = calculate_dx_dy_arrow(x, y, orient, 1, 1)
            patch.append(plt.arrow(x, y, dx, dy, color='gold', width=0.5, shape='full'))
            
        # Home players' direction
        for x, y, direction, speed in zip(homeX, homeY, homeDir, homeSpeed):
            dx, dy = calculate_dx_dy_arrow(x, y, direction, speed, 1)
            patch.append(plt.arrow(x, y, dx, dy, color='black', width=0.25, shape='full'))
        
        # Home players' location
        awayX = playAway.query('time == ' + str(time))['x']
        awayY = playAway.query('time == ' + str(time))['y']
        awayNum = playAway.query('time == ' + str(time))['jerseyNumber']
        awayOrient = playAway.query('time == ' + str(time))['o']
        awayDir = playAway.query('time == ' + str(time))['dir']
        awaySpeed = playAway.query('time == ' + str(time))['s']
        patch.extend(plt.plot(awayX, awayY, 'o',c='orangered', ms=20, mec='white'))
        
        # Away players' jersey number 
        for x, y, num in zip(awayX, awayY, awayNum):
            patch.append(plt.text(x, y, int(num), va='center', ha='center', color='white', size='medium'))
            
        # Away players' orientation
        for x, y, orient in zip(awayX, awayY, awayOrient):
            dx, dy = calculate_dx_dy_arrow(x, y, orient, 1, 1)
            patch.append(plt.arrow(x, y, dx, dy, color='orangered', width=0.5, shape='full'))
        
        # Away players' direction
        for x, y, direction, speed in zip(awayX, awayY, awayDir, awaySpeed):
            dx, dy = calculate_dx_dy_arrow(x, y, direction, speed, 1)
            patch.append(plt.arrow(x, y, dx, dy, color='black', width=0.25, shape='full'))
        
        # Away players' location
        footballX = playFootball.query('time == ' + str(time))['x']
        footballY = playFootball.query('time == ' + str(time))['y']
        patch.extend(plt.plot(footballX, footballY, 'o', c='black', ms=10, mec='white', data=playFootball.query('time == ' + str(time))['team']))
        
        
        return patch
    
    ims = [[]]
    for time in np.arange(minTime, maxTime+1):
        patch = update_animation(time)
        ims.append(patch)
        
    anim = animation.ArtistAnimation(fig, ims, repeat=False)
    
    return anim

In [ ]:
anim = animate_player_movement(1, 75, 2018090600)

In [ ]:
writer = FFMpegWriter(fps=10)
anim.save('animation_notrail.mp4', writer=writer)

In [ ]:
Video("animation_notrail.mp4")

In [ ]:
anim = animate_player_movement(1, 4192, 2018091001)

In [ ]:
writer = FFMpegWriter(fps=10)
anim.save('animation_notrail.mp4', writer=writer)

In [ ]:
Video("animation_notrail.mp4")

In [ ]:
fig, ax = create_football_field()
Week1.query("playId == 4192 and team == 'away'") \
    .plot(x='x', y='y', kind='scatter', ax=ax, color='orange', s=30, legend='Away')
Week1.query("playId == 4192 and team == 'home'") \
    .plot(x='x', y='y', kind='scatter', ax=ax, color='blue', s=30, legend='Home')
plt.title('Play # 4192')
plt.legend()
plt.show()

In [ ]:
Week1.tail()